# *Task* - Cloud-Based Neural Network Inference

## Summary

Implement and evaluate two different methodologies to moving neural network inference from automated vehicles to connected cloud servers.

- [Background and Motivation](#background-and-motivation)
- [Task](#task)
- [Required Tools and Data](#required-tools-and-data)
- [Hints](#hints)

## Background and Motivation

Neural network inference is a time- and energy-consuming task, with state-of-the-art models growing ever more complex. Current (5G) and future (6G) communication technologies promise to offer high-bandwidth, low-latency communication. Both these trends suggest to off-load neural network inference (and possibly other driving functions) from running on automated vehicles to connected cloud servers.

This task is about evaluating two possible methodologies for cloud-based inference against each other. Semantic image segmentation is considered as the exemplary application.
1. Equivalent ROS-based setup on both vehicle and cloud server with MQTT-based transmission of ROS messages
   - vehicle camera publishes images on vehicle's ROS network;
   - MQTT bridge on vehicle sends ROS image messages via MQTT broker to MQTT bridge on cloud server, where images are again available as ROS messages;
   - ROS node for neural network inference on cloud server computes semantic segmentation image using a trained model;
   - MQTT bridge on cloud server sends ROS segmentation image messages via MQTT broker to MQTT bridge on vehicle, where segmentation images are again available as ROS messages.
1. Cloud-server acting as inference server only using [TensorFlow Serving RESTful API](https://www.tensorflow.org/tfx/serving/api_rest)
   - vehicle camera publishes images on vehicle's ROS network;
   - cloud server is serving a trained model via TensorFlow Serving RESTful API;
   - ROS node on vehicle pre-processes the camera image for a call to the cloud's RESTful API;
   - ROS node on vehicle post-processes the API's response (model output) to publish the semantic segmentation image.

## Task

The task is to implement and evaluate two different methodologies to moving neural network inference from automated vehicles to connected cloud servers:
1. Equivalent ROS-based setup on both vehicle and cloud server with MQTT-based transmission of ROS messages
1. Cloud-server acting as inference server only using [TensorFlow Serving RESTful API](https://www.tensorflow.org/tfx/serving/api_rest)

### Subtasks

> ***Note:*** *The subtasks listed below do not have to be followed strictly. They serve the purpose of guiding you along your own research for this topic.*

1. Research other possible approaches for cloud-based neural network inference in the context of automated driving.
1. Set up a working cloud inference using the MQTT-based approach.
   1. launch an MQTT broker;
   1. set up and launch an MQTT bridge on one computer to send image messages to a second computer and receive segmented image messages;
   1. set up and launch an MQTT bridge on a second computer to receive image messages and send back segmented image messages;
   1. launch an image segmentation inference ROS node on the second computer;
   1. implement and launch a ROS node on the first computer to publish image messages and subscribe segmented image messages.
1. Set up a working cloud inference using the TensorFlow Serving approach.
   1. research how to configure and launch a model server using [TensorFlow Serving RESTful API](https://www.tensorflow.org/tfx/serving/api_rest) (preferably as a Docker container);
   1. launch the model server on the second computer;
   1. implement and launch a ROS node on the first computer to publish image messages and subscribe segmented image messages;
   1. implement and launch a ROS node on the first computer to subscribe image messages, prepare REST API calls, make such a call to the model server on the second computer, post-process the model server's response, and publish final segmented image messages.
1. Extend image publisher / segmentation subscriber ROS node with functionality to measure elapsed runtime of the cloud inference process, including communication.
1. Evaluate both approaches against each other with regard to runtimes, transmitted data sizes and other advantages/disadvantages on either side.
1. Document your research, developed approach, and evaluations in a Jupyter notebook report. Explain and reproduce individual parts of your implemented functions with exemplary data.

## Required Tools and Data

### Tools

- ROS
- MQTT bridge ROS node *(see [ACDC Exercise: Cloud Based Object Fusion](https://github.com/ika-rwth-aachen/acdc/wiki/Section-5-Cloud-Based-Object-Fusion))*
- MQTT broker *(see [Hints](#hints))*
- ROS image segmentation inference node *(see [ACDC Exercise: Semantic Image Segmentation](https://github.com/ika-rwth-aachen/acdc/wiki/Section-2-Semantic-Image-Segmentation))*
- TensorFlow
- [TensorFlow Serving RESTful API](https://www.tensorflow.org/tfx/serving/api_rest)

### Data

- trained image segmentation neural network *(see model weights in [ACDC Exercise: Semantic Image Segmentation](https://github.com/ika-rwth-aachen/acdc/wiki/Section-2-Semantic-Image-Segmentation))*
- exemplary camera images for image segmentation *(see downloadable bag file from [ACDC Exercise: Semantic Image Segmentation](https://github.com/ika-rwth-aachen/acdc/wiki/Section-2-Semantic-Image-Segmentation))*

## Hints

### Relevant ACDC Sections

- **Sensor Data Processing Algorithms**
  - Image Segmentation
- **Connected Driving**
  - Collective Cloud Functions

### Computer Setup

Instead of having to use two separate computers (one to represent the vehicle, one for the cloud server), the methodology can also be developed entirely on a single machine. The components of the data flow stay the same, but all components are running on the same computer. For the MQTT approach this means that only one MQTT bridge is running, which is sending data to itself (e.g., data on ROS topic `/vehicle/image` is sent to the MQTT broker, received by the same MQTT bridge, and re-published on the same computer on ROS topic `/cloud/image`).

It is also okay to evaluate the two methodologies using the single-computer approach. The task focuses on performance differences due to different ways of model inference, not on measuring the actual communication latencies.

### MQTT Broker

The MQTT broker software [*Mosquitto*](https://hub.docker.com/_/eclipse-mosquitto) can easily be launched as a Docker container.

```
docker run --rm -d -p 1883:1883 --name mosquitto eclipse-mosquitto
```

### Trained Neural Network Weights

The trained image segmentation models from [ACDC Exercise: Semantic Image Segmentation](https://github.com/ika-rwth-aachen/acdc/wiki/Section-2-Semantic-Image-Segmentation) (e.g. `mobilenet_v3_large_968_608_os8.pb`) are shared as *TensorFlow frozen graphs*. Please let us know if you encounter problems serving this format with TensorFlow Serving. We could then potentially export the models in the newer *TensorFlow SavedModel* format.